In [1]:
import openke
import torch
import torch.autograd
import time
import random
from openke.config import Trainer, Tester
from openke.module.model import TransE
from openke.module.loss import MarginLoss
from openke.module.strategy import NegativeSampling
from openke.data import TrainDataLoader, TestDataLoader

In [131]:
def filter_triples(input_entity_file, input_triple_file, output_triple_file, remove_num=500):
    with open(input_entity_file, 'r') as f:
        total_entities = int(f.readline().strip())

    selected_indices = set(random.sample(range(total_entities), remove_num))

    with open(input_triple_file, 'r') as f:
        lines = f.readlines()

    header = lines[0]  
    triples = lines[1:]  

    filtered_triples = []
    for triple in triples:
        h, t, r = map(int, triple.split())
        if h not in selected_indices and t not in selected_indices:
            filtered_triples.append(triple)

    with open(output_triple_file, 'w') as f:
        f.write(str(len(filtered_triples)) + '\n') 
        f.writelines(filtered_triples)  

def filter_relations(input_relation_file, input_type_file, input_triple_file, output_triple_file, remove_num=5):
    # 读取关系文件以获取关系的编号
    with open(input_relation_file, 'r') as f:
        f.readline()  # Skip the first line which contains the total number of relations
        relations = [line.strip().split()[-1] for line in f]  # Assume relation ID is the last column

    # 随机选择要删除的关系编号
    selected_relations = set(random.sample(relations, remove_num))

    # 读取类型文件并为每个选定的关系收集类型编号
    relation_types = set()
    with open(input_type_file, 'r') as f:
        f.readline()  # Skip the first line with the total number of relations
        for line in f:
            parts = line.strip().split()
            if parts[0] in selected_relations:  # Assume relation ID is the second column in this file
                relation_types.update(parts[1:])  # Add all type IDs related to the relation

    # 读取三元组文件，并过滤掉包含选定类型的三元组
    with open(input_triple_file, 'r') as f:
        lines = f.readlines()

    header = lines[0]  # Header with the total count of triples, which we ignore
    triples = lines[1:]  # Actual triples start from the second line

    filtered_triples = []
    for triple in triples:
        _, t, _ = triple.split()  # Assume type ID is the second column in this file
        if t not in relation_types:
            filtered_triples.append(triple)

    # 将过滤后的三元组写入输出文件
    with open(output_triple_file, 'w') as f:
        f.write(str(len(filtered_triples)) + '\n')  # Write the count of remaining triples
        f.writelines(filtered_triples)  # Write the triples
        

def calculate_gradients(model, data):
    model.eval()  

    loss = model.model({
        'batch_h': torch.autograd.Variable(torch.from_numpy(data['batch_h']).cuda()),
        'batch_t': torch.autograd.Variable(torch.from_numpy(data['batch_t']).cuda()),
        'batch_r': torch.autograd.Variable(torch.from_numpy(data['batch_r']).cuda()),
        'batch_y': torch.autograd.Variable(torch.from_numpy(data['batch_y']).cuda()),
        'mode': data['mode']
    })
    loss_scalar = torch.mean(loss)
    params = filter(lambda p: p.requires_grad, model.parameters())
    params_to_update = [param for name, param in model.named_parameters() if name.endswith('.weight')]
    return torch.autograd.grad(loss_scalar, params_to_update, create_graph=True)

def hvps(grad_all, model_params, h_estimate):
    element_product = 0
    for grad_elem, v_elem in zip(grad_all, h_estimate):
        element_product += torch.sum(grad_elem * v_elem)
    return_grads = torch.autograd.grad(element_product, model_params, create_graph=True)
    
    return return_grads
    
def GIF_unleanring(model, train_dataloader, test_dataloader, iteration=1, damp=0.0, scale=50):
    start_time = time.time()

    for data in train_dataloader:
        grad_full = calculate_gradients(model, data)

    for data in test_dataloader:
        grad_removed = calculate_gradients(model, data)

    grad1 = [g1 - g2 for g1, g2 in zip(grad_full, grad_removed)]
    grad2 = grad_removed
    res_tuple = (grad_full, grad1, grad2)

    v = tuple(grad1 - grad2 for grad1, grad2 in zip(res_tuple[1], res_tuple[2]))
    h_estimate = tuple(grad1 - grad2 for grad1, grad2 in zip(res_tuple[1], res_tuple[2]))
    
    for _ in range(iteration):
        model_params  = [p for p in model.parameters() if p.requires_grad]
        hv = hvps(res_tuple[0], model_params, h_estimate)
        with torch.no_grad():
            h_estimate = [ v1 + (1-damp)*h_estimate1 - hv1/scale for v1, h_estimate1, hv1 in zip(v, h_estimate, hv)]
            
    params_change = [h_est / scale for h_est in h_estimate]
    params_esti   = [p1 + p2 for p1, p2 in zip(params_change, model_params)]

    print(time.time() - start_time)
    
    return params_esti

def update_and_save_checkpoint(checkpoint_path, new_checkpoint_path, new_params):
    weights = torch.load(checkpoint_path)
    weights['ent_embeddings.weight'] = new_params[0]
    weights['rel_embeddings.weight'] = new_params[1]
    torch.save(weights, new_checkpoint_path)
    print(f"Updated checkpoint saved to {new_checkpoint_path}")

In [195]:
# removed_entities = 3000
# removed_files = f'./benchmarks/FB15K237/removed_{removed_entities}_train2id.txt'
# filter_triples('./benchmarks/FB15K237/entity2id.txt', './benchmarks/FB15K237/train2id.txt', removed_files, removed_entities)
removed_relations = 9
removed_files = f'./benchmarks/FB15K237/removed_{removed_relations}_train2id.txt'
filter_relations(input_relation_file='./benchmarks/FB15K237/relation2id.txt', 
                 input_type_file='./benchmarks/FB15K237/type_constrain.txt',
                 input_triple_file='./benchmarks/FB15K237/train2id.txt',
                 output_triple_file = removed_files, remove_num=removed_relations)

In [196]:
retrain_dataloader = TrainDataLoader(
	in_path = None, 
    tri_file = removed_files,
    ent_file = "./benchmarks/FB15K237/entity2id.txt",
    rel_file = "./benchmarks/FB15K237/relation2id.txt",
	nbatches = 100,
	threads = 8, 
	sampling_mode = "normal", 
	bern_flag = 1, 
	filter_flag = 1, 
	neg_ent = 25,
	neg_rel = 0)

Training Files Path : ./benchmarks/FB15K237/removed_9_train2id.txt
Entity Files Path : ./benchmarks/FB15K237/entity2id.txt
Relation Files Path : ./benchmarks/FB15K237/relation2id.txt
The toolkit is importing datasets.
The total of relations is 237.
The total of entities is 14541.


In [197]:
test_dataloader = TestDataLoader("./benchmarks/FB15K237/", "link")

The total of train triples is 221600.
Input Files Path : ./benchmarks/FB15K237/
The total of test triples is 20466.
The total of valid triples is 17535.


In [198]:
retrain_transe = TransE(
	ent_tot = retrain_dataloader.get_ent_tot(),
	rel_tot = retrain_dataloader.get_rel_tot(),
	dim = 200, 
	p_norm = 1, 
	norm_flag = True)

In [199]:
retrain_model = NegativeSampling(
	model = retrain_transe, 
	loss = MarginLoss(margin = 5.0),
	batch_size = retrain_dataloader.get_batch_size()
)

In [200]:
retrain_trainer = Trainer(model = retrain_model, data_loader = retrain_dataloader, train_times = 1000, alpha = 1.0, use_gpu = True)
retrain_trainer.run()
retrain_transe.save_checkpoint(f'./checkpoint/retrain_{removed_relations}_transe.ckpt')

Finish initializing...


Epoch 999 | loss: 1.770634: 100%|███████████████████████████████████████████████████| 1000/1000 [13:15<00:00,  1.26it/s]


In [201]:
retrain_transe.load_checkpoint(f'./checkpoint/retrain_{removed_relations}_transe.ckpt')
retrain_tester = Tester(model = retrain_transe, data_loader = test_dataloader, use_gpu = True)
retrain_tester.run_link_prediction(type_constrain = False)

100%|███████████████████████████████████████████████████████████████████████████| 20466/20466 [00:17<00:00, 1200.60it/s]

0.41864556074142456


(0.24929894506931305,
 477.66607666015625,
 0.41864556074142456,
 0.28249290585517883,
 0.16212254762649536)

In [202]:
train_dataloader = TrainDataLoader(
	in_path = None, 
    tri_file = "./benchmarks/FB15K237/train2id.txt",
    ent_file = "./benchmarks/FB15K237/entity2id.txt",
    rel_file = "./benchmarks/FB15K237/relation2id.txt",
	nbatches = 100,
	threads = 8, 
	sampling_mode = "normal", 
	bern_flag = 1, 
	filter_flag = 1, 
	neg_ent = 25,
	neg_rel = 0)
print('----------------------------------------------------------------------')
retrain_dataloader = TrainDataLoader(
	in_path = None, 
    tri_file = removed_files,
    ent_file = "./benchmarks/FB15K237/entity2id.txt",
    rel_file = "./benchmarks/FB15K237/relation2id.txt",
	nbatches = 100,
	threads = 8, 
	sampling_mode = "normal", 
	bern_flag = 1, 
	filter_flag = 1, 
	neg_ent = 25,
	neg_rel = 0)

no type constraint results:
metric:			 MRR 		 MR 		 hit@10 	 hit@3  	 hit@1 
l(raw):			 0.077275 	 733.083191 	 0.186260 	 0.072462 	 0.025652 
r(raw):			 0.205607 	 462.685638 	 0.383807 	 0.222125 	 0.120004 
averaged(raw):		 0.141441 	 597.884399 	 0.285034 	 0.147293 	 0.072828 

l(filter):		 0.169097 	 514.968506 	 0.324098 	 0.188557 	 0.092886 
r(filter):		 0.329501 	 440.363617 	 0.513193 	 0.376429 	 0.231359 
averaged(filter):	 0.249299 	 477.666077 	 0.418646 	 0.282493 	 0.162123 
0.418646
Training Files Path : ./benchmarks/FB15K237/train2id.txt
Entity Files Path : ./benchmarks/FB15K237/entity2id.txt
Relation Files Path : ./benchmarks/FB15K237/relation2id.txt
The toolkit is importing datasets.
The total of relations is 237.
The total of entities is 14541.
----------------------------------------------------------------------
The total of train triples is 272115.
Training Files Path : ./benchmarks/FB15K237/removed_9_train2id.txt
Entity Files Path : ./benchmarks/FB15K237/enti

In [203]:
model = TransE(
	ent_tot = train_dataloader.get_ent_tot(),
	rel_tot = train_dataloader.get_rel_tot(),
	dim = 200, 
	p_norm = 1, 
	norm_flag = True)
model.to('cuda')
model.load_checkpoint('./checkpoint/transe.ckpt')
model = NegativeSampling(
	model = model, 
	loss = MarginLoss(margin = 5.0),
	batch_size = train_dataloader.get_batch_size()
)

The total of train triples is 221600.


In [204]:
params_esti = GIF_unleanring(model, train_dataloader, retrain_dataloader, iteration=100, damp=0.0, scale=50)

3.467162609100342


In [205]:
update_and_save_checkpoint(checkpoint_path='./checkpoint/transe.ckpt', 
                           new_checkpoint_path=f'./checkpoint/GIF_{removed_relations}_TransE.ckpt', 
                           new_params=params_esti)

Updated checkpoint saved to ./checkpoint/GIF_9_TransE.ckpt


In [206]:
# dataloader for test
test_dataloader = TestDataLoader("./benchmarks/FB15K237/", "link")

# define the model
unlearn_transe = TransE(
	ent_tot = train_dataloader.get_ent_tot(),
	rel_tot = train_dataloader.get_rel_tot(),
	dim = 200, 
	p_norm = 1, 
	norm_flag = True)

# test the model
unlearn_transe.load_checkpoint(f'./checkpoint/GIF_{removed_entities}_TransE.ckpt')
unlearn_tester = Tester(model = unlearn_transe, data_loader = test_dataloader, use_gpu = True)
unlearn_tester.run_link_prediction(type_constrain = False)

Input Files Path : ./benchmarks/FB15K237/
The total of test triples is 20466.
The total of valid triples is 17535.


100%|███████████████████████████████████████████████████████████████████████████| 20466/20466 [00:16<00:00, 1244.84it/s]

0.43968045711517334


(0.2585415840148926,
 255.31497192382812,
 0.43968045711517334,
 0.2900664508342743,
 0.1673995852470398)

no type constraint results:
metric:			 MRR 		 MR 		 hit@10 	 hit@3  	 hit@1 
l(raw):			 0.088247 	 569.296326 	 0.206049 	 0.086680 	 0.032200 
r(raw):			 0.247412 	 167.087662 	 0.436871 	 0.269032 	 0.155135 
averaged(raw):		 0.167830 	 368.191986 	 0.321460 	 0.177856 	 0.093668 

l(filter):		 0.171295 	 363.839294 	 0.329620 	 0.191293 	 0.093179 
r(filter):		 0.345789 	 146.790634 	 0.549741 	 0.388840 	 0.241620 
averaged(filter):	 0.258542 	 255.314972 	 0.439680 	 0.290066 	 0.167400 
0.439680
